# Testing isostatic boundary conditions in UW2

This is a check to make sure UW2's pressure boundary condition properly works.

The setup is:

     <--------- 100 km ------------>
    _________________________________
    |                               |  ^
    |                               |  |
    |     compressible air          |  50 km 
    |        1 kg.m^-3              |  |
    |                               |  | 
    |_______________________________|  v 
    |        crust                  |  ^ 
    |        500 kg.m^-3            |  25 km 
    |_______________________________|  v 
    |        mantle                 |  ^ 
    |        1000 kg.m^-3           |  25 km 
    |_______________________________|  v 

Now with sedimentation! Sedimentation starts at 0 km, so we should expect
that the mantle level stays at approximately 25 km thick the whole time,
even as the entire crust is replaced

Otherwise, same as before:
We pull each vertical wall at 1 cm/yr (total extension is 2 cm/yr)
The top wall has a free slip boundary (no air can flow in or out of the top)
The bottom wall as a pressure boundary condition

Note: the model does encounter some velocity instabilities, but broadly seems to be correct.
With higher resolution, or smaller timesteps, it might be happier

In [ ]:
import UWGeodynamics as GEO
from UWGeodynamics.surfaceProcesses import SedimentationThreshold
import numpy as np

In [ ]:
u = GEO.UnitRegistry
GEO.rcParams['solver'] = "mumps"
GEO.rcParams["surface.pressure.normalization"] = True
GEO.rcParams["nonlinear.tolerance"] = 1e-3
GEO.rcParams["initial.nonlinear.tolerance"] = 1e-3
GEO.rcParams["minimum.viscosity"] = 1e18 * u.pascal * u.second
GEO.rcParams["maximum.viscosity"] = 1e24 * u.pascal * u.second
GEO.rcParams["output.directory"] = "simple_isos_3"

In [ ]:
# Characteristic values of the system
half_rate = 1.0 * u.centimeter / u.year
model_length = 100e3 * u.meter
model_height = 100e3 * u.meter
refViscosity = 1e22 * u.pascal * u.second
bodyforce = 500 * u.kilogram / u.metre**3 * 9.81 * u.meter / u.second**2

KL = model_length
Kt = KL / half_rate
KM = bodyforce * KL**2 * Kt**2

GEO.scaling_coefficients["[length]"] = KL
GEO.scaling_coefficients["[time]"] = Kt
GEO.scaling_coefficients["[mass]"]= KM

In [ ]:
Model = GEO.Model(elementRes=(100,100), 
                  minCoord=(-50 * u.kilometer, -50 * u.kilometer),
                  maxCoord=( 50 * u.kilometer,  50 * u.kilometer))

In [ ]:
air = Model.add_material(name="air", shape=GEO.shapes.Layer2D(top=Model.top, bottom=0.0))
uc =  Model.add_material(name="crust",  shape=GEO.shapes.Layer2D(top=air.bottom, bottom=-25*u.kilometer))
mantle = Model.add_material(name="mantle",  shape=GEO.shapes.Layer2D(top=uc.bottom, bottom=Model.bottom))
sediment          = Model.add_material(name="sediment")

air.density       =    1. * u.kilogram / u.metre**3
sediment.density  =  500. * u.kilogram / u.metre**3
uc.density        =  500. * u.kilogram / u.metre**3
mantle.density    = 1000. * u.kilogram / u.metre**3



air.viscosity      = 1e18 * u.pascal * u.second
uc.viscosity       = 1e21 * u.pascal * u.second  # make this slightly higher so UW has something to do
sediment.viscosity       = 1e21 * u.pascal * u.second  # make this slightly higher so UW has something to do
mantle.viscosity   = 1e20 * u.pascal * u.second


# we need to make the air compressible, so we can put a free slip condition on the top
# of the model
air.compressibility = 1e3  # Not sure what's a good value

In [ ]:
Model.surfaceProcesses = GEO.surfaceProcesses.SedimentationThreshold(air=[air], sediment=[sediment], threshold=0.*u.kilometer)

In [ ]:
# This is where isostasy is imposed. We first calculate the lithostatic pressure along the entire
# bottom of the model. 
P, bottomPress = Model.get_lithostatic_pressureField()
# Then, since the model is laterally homogenous, we average it, and put it into megapascals
bottomPress = GEO.Dimensionalize(np.average(bottomPress), u.megapascal)

Model.set_velocityBCs(left=[-1.0 * u.centimetre / u.year,None], 
                      right=[1.0 * u.centimetre / u.year,None], 
                      top=[None, 0. * u.centimetre / u.year],  # notice the top is free slip
                      # This is where the pressure boundary is applied.
                      bottom=[None,bottomPress])

In [ ]:
Model.init_model()
Model.solve()

Model.run_for(20e6* u.year, dt=10e3*u.years)